In [2]:
import globus_sdk

In [8]:
''' Code to get access token'''

CLIENT_ID = '7a71f6ff-14d4-4e3a-8dbf-0e23a573f519' # this can be obtained from Apps and Services -> Projects -> Client ID 

client = globus_sdk.NativeAppAuthClient(CLIENT_ID)
client.oauth2_start_flow()

authorize_url = client.oauth2_get_authorize_url()
print('Please go to this URL and login: {0}'.format(authorize_url))

# this is to work on Python2 and Python3 -- you can just use raw_input() or
# input() for your specific version
get_input = getattr(__builtins__, 'raw_input', input)
auth_code = get_input(
    'Please enter the code you get after login here: ').strip()
token_response = client.oauth2_exchange_code_for_tokens(auth_code)

globus_auth_data = token_response.by_resource_server['auth.globus.org']
globus_transfer_data = token_response.by_resource_server['transfer.api.globus.org']

# most specifically, you want these tokens as strings
AUTH_TOKEN = globus_auth_data['access_token']
TRANSFER_TOKEN = globus_transfer_data['access_token']

Please go to this URL and login: https://auth.globus.org/v2/oauth2/authorize?client_id=7a71f6ff-14d4-4e3a-8dbf-0e23a573f519&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=openid+profile+email+urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=6hB7KldBBrw6XkRPOfu2Tu3QDumWIHR7Aqv4SgrH-5U&code_challenge_method=S256&access_type=online
Please enter the code you get after login here: XNqRXwiGCpyRMsRDeBoQdG199ytZfH


In [9]:
# a GlobusAuthorizer is an auxiliary object we use to wrap the token. In
# more advanced scenarios, other types of GlobusAuthorizers give us
# expressive power
authorizer = globus_sdk.AccessTokenAuthorizer(TRANSFER_TOKEN)
tc = globus_sdk.TransferClient(authorizer=authorizer)

# high level interface; provides iterators for list responses
print("My Endpoints:")
for ep in tc.endpoint_search(filter_scope="recently-used"):
    print("[{}] {}".format(ep["id"], ep["display_name"]))

My Endpoints:
[606579ae-5b03-11e9-bf32-0edbf3a4e7ee] cac_dtn_test
[7167cb38-9f78-11e6-b0dd-22000b92c261] Cheaha cluster off-campus (UAB Science DMZ)
[9c8c88c2-ea4a-11e6-b9ba-22000b9a448b] Cheaha cluster on-campus (UAB Science DMZ)
[db57ddde-6d04-11e5-ba46-22000b92c6ec] ESnet Read-Only Test DTN at Sunnyvale
[ff3045ea-a112-11ea-8f07-0a21f750d19b] Work Mac


In [15]:
destination_endpoint_id = "9c8c88c2-ea4a-11e6-b9ba-22000b9a448b"
source_endpoint_id = "606579ae-5b03-11e9-bf32-0edbf3a4e7ee"
tdata = globus_sdk.TransferData(tc, source_endpoint_id,
                                 destination_endpoint_id,
                                 label="Dataset test",
                                sync_level=None, verify_checksum=True)
tdata.add_item("/datasets/ds04", "/data/user/mmoo97/TEST_TRANSFER/",
                recursive=True)
# tdata.add_item("/source/path/file.txt",
#                "/dest/path/file.txt")
transfer_result = tc.submit_transfer(tdata)
print("task_id =", transfer_result["task_id"])

task_id = 37948d26-032c-11eb-81a3-0e2f230cc907


In [19]:
task = tc.get_task(transfer_result["task_id"])
if task["status"] == "SUCCEEDED":
    print("Task Status: " + task["status"])
    print("Effective Speed: " + str(round(task["effective_bytes_per_second"]/(1024*1024), 2)) + " MB/s")
else:
    print("Task Status: " + task["status"])

Task Status: SUCCEEDED
Effective Speed: 186.71 MB/s
